In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
import torch

In [ ]:
torch.tensor([1,2,3], device=2).tolist()

In [38]:
import torch


tensor([[2, 1],
        [1, 2],
        [1, 0]])

tensor([3, 1, 2, 3, 2, 1, 2, 1, 4, 0, 1, 2, 2, 3, 2, 4, 4, 0, 2, 1, 0, 2, 3, 3,
        0, 4, 1, 1, 4, 1, 1, 2, 4, 2, 3, 2, 4, 1, 0, 1, 0, 4, 0, 0, 1, 3, 0, 1,
        4, 4, 2, 0, 4, 0, 1, 0, 2, 3, 2, 3, 4, 1, 4, 4, 4, 4, 2, 4, 4, 2, 4, 1,
        2, 0, 3, 3, 0, 2, 4, 2, 0, 0, 1, 0, 3, 2, 4, 3, 2, 3, 1, 2, 1, 2, 2, 0,
        2, 4, 4, 4])

In [44]:
next(iter(loader))


[tensor([[0.3056, 0.7637, 0.1714, 0.4287, 0.3613, 0.2151, 0.6616, 0.3501, 0.4148,
          0.8312, 0.0177, 0.6140, 0.6437, 0.9547, 0.6284, 0.2689, 0.7776, 0.1712,
          0.1284, 0.3514, 0.6666, 0.4133, 0.0975, 0.0073, 0.7258, 0.5191, 0.4540,
          0.5046, 0.7067, 0.0560, 0.5030, 0.8775, 0.8641, 0.9416, 0.0441, 0.7003,
          0.5934, 0.3533, 0.6483, 0.2522, 0.5404, 0.3404, 0.3893, 0.4462, 0.6699,
          0.0229, 0.2470, 0.0351, 0.9948, 0.5545]]),
 tensor([1])]

In [11]:
import torch
torch.rand((1000, 5))

tensor([[0.0881, 0.9478, 0.1547, 0.2581, 0.1471],
        [0.2729, 0.6230, 0.3299, 0.8251, 0.2420],
        [0.6818, 0.8793, 0.9149, 0.5630, 0.1598],
        ...,
        [0.7641, 0.4122, 0.3678, 0.6534, 0.0630],
        [0.7970, 0.5063, 0.6189, 0.4730, 0.8577],
        [0.2987, 0.1672, 0.3231, 0.0253, 0.0268]])

In [2]:
import os
import pandas as pd
from ordinaloss.utils.pretrained_models import (classification_model_vgg, 
                                                DummyModel)

from ordinaloss.utils.data_utils import data_load

from ordinaloss.utils.loss_utils import CSCELoss
from ordinaloss.utils.loss_utils import create_ordinal_cost_matrix
from ordinaloss.utils.loss_utils import PredictionLoss, CombinedLoss

from ordinaloss.nextgen_engine import model_engine
from ordinaloss.utils.basic_utils import satisfy_constraints, modify_lambdas


import matplotlib.pyplot as plt

from torch.optim import SGD
import torch

# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# device = "cuda:1" if torch.cuda.is_available() else "cpu"
device = "cuda:2" if torch.cuda.is_available() else "cpu"
#device = "cuda:3" if torch.cuda.is_available() else "cpu"

/home/dsi/itaym/anaconda3/envs/maruloss/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loaded ordinaloss.utils.pretrained_models
loaded ordinaloss.utils.data_utils
loaded ordinaloss.utils.loss_utils
loaded ordinaloss.utils.metric_utils!!
loaded ordinaloss.nextgen_engine.model_engine 


In [4]:
loaders, _, _ = data_load("../datasets/kneeKL224/", 16)
# out = []
# for Xb, yb, in loaders["test"]:
#     out.append(yb)
# pd.Series(torch.concat(out).numpy()).value_counts(normalize=True, sort=False)

In [1]:
import numpy as np

In [5]:
np.bincount([1,2,3], minlength=7)

array([0, 1, 1, 1, 0, 0, 0])

In [9]:
META_LEARNING_RATE = 10
LEARNING_RATE = 1.0e-3
WEIGHT_DECAY = 5.0e-2
COST_DISTANCE = 3
DIAGONAL_VALUE = 20
PATIENCE = 3
MIN_DELTA = 0.99

#model = DummyModel()
model = classification_model_vgg("vgg16", num_classes = 5)

engine = model_engine.OrdinalEngine(
    model=model, loss_fn=None, #The loss function would be modified soon.
    device=device, loaders=loaders, n_classes=5,
    use_lr_scheduler=False, callbacks=[],
    optimizer_fn=SGD, lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

#constraints = torch.tensor([0.2,1,1,1,1], device = device)
constraints = torch.tensor([1,1,1,0.05,1], device = device) 

#current_lambdas = torch.tensor([0.0, 0.0, 0.0, 0.0, 0.2], device=device)
current_lambdas = torch.zeros(size=(5,), device=device)

cost_matrix = create_ordinal_cost_matrix(5, cost_distance=COST_DISTANCE, diagonal_value=DIAGONAL_VALUE)

while True:

    print(f"training model with {current_lambdas}")
    csce_loss = CSCELoss(cost_matrix)
    prediction_loss = PredictionLoss(lambdas = current_lambdas)

    loss_fn = CombinedLoss(csce_loss, prediction_loss)
    engine.set_loss_fn(loss_fn) #The loss is assigned again

    engine.train_until_converge(50, patience=PATIENCE, min_delta=MIN_DELTA)
    test_dist = engine.predict_dist_on_test()
    print(f"test_dist: {test_dist}")

    if not satisfy_constraints(test_dist, constraints):
        #modify loss function
        current_lambdas = modify_lambdas(constraints, test_dist, current_lambdas, META_LEARNING_RATE).to(device=device)
        print("Lambdas modified!")

    else:
        break

training model with tensor([0., 0., 0., 0., 0.], device='cuda:2')


Evaluating...: 100%|██████████| 52/52 [00:07<00:00,  6.69it/s, accuracy=0.429, loss=0.676]


inf
0.6762158198160352


Evaluating...: 100%|██████████| 52/52 [00:02<00:00, 18.87it/s, accuracy=0.454, loss=0.633]


0.6762158198160352
0.6331482981365472


...

Evaluating...: 100%|██████████| 52/52 [00:04<00:00, 10.58it/s, accuracy=0.563, loss=0.626]


0.556422978567442
0.625518301353039
strike 1! didn't increase by mindelta.


Evaluating...: 100%|██████████| 52/52 [00:05<00:00, 10.09it/s, accuracy=0.32, loss=0.908] 

0.556422978567442
0.9081504690589397
strike 2! didn't increase by mindelta.
model converges


test_dist: tensor([0.0471, 0.0048, 0.8225, 0.0000, 0.1256], device='cuda:2')


In [23]:
constraints = torch.tensor([0.2, 1, 1, 1, 1])
test_dist   = torch.tensor([0.6, 0, 0, 0, 0.1])
current_lambdas = torch.ones(5)*0.2

modify_lambdas(constraints, test_dist, current_lambdas)

current_lambdas tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])
new_lambdas tensor([0.3846, 0.1538, 0.1538, 0.1538, 0.1538])


tensor([0.3846, 0.1538, 0.1538, 0.1538, 0.1538])

In [ ]:
#model = classification_model_vgg("vgg16", num_classes = 5)

cost_matrix = create_ordinal_cost_matrix(5, cost_distance=3, diagonal_value=20)



#callbacks = [PrintingCallback()]
callbacks = []

engine = model_engine.OrdinalEngine(
    model=model, loss_fn=CSCELoss(cost_matrix), 
    device=device, loaders=loaders, n_classes=5,
    use_lr_scheduler=False, callbacks=callbacks,
    optimizer_fn=SGD, lr=1.0e-3, weight_decay=5.0e-2)

loaded ordinaloss.nextgen_engine.model_engine


In [127]:
cost_matrix = create_ordinal_cost_matrix(5, cost_distance=3, diagonal_value=20)


tensor([1., 1., 1., 1., 1.])

In [4]:
constraints = torch.tensor([0.2, 1, 1, 1, 1])
test_dist   = torch.tensor([0.8, 0, 0, 0, 0.1])
current_lambdas = torch.ones(5)*0.2

In [137]:


constraints = torch.tensor([0.2, 1, 1, 1, 1])
test_dist   = torch.tensor([0.8, 0, 0, 0, 0.1])
current_lambdas = torch.ones(5)*0.2


modify_loss_function(constraints, test_dist, current_lambdas)

tensor([0.5000, 0.1250, 0.1250, 0.1250, 0.1250])